# Labbuppgift: mäta Z-bosonens massa

## 1.1 Introduktion
   
Börja med att läsa igenom introduktionsmaterialet om ATLAS-experimentet och hur dess detektorer används för att mäta och rekonstruera vad som händer i proton-proton-kollisionerna som Large Hadron Collider skapar. Om inte [notebook 1a](1a-Introduction-information.ipynb) fungerar så finns informationen även som pdf [här](images/1-Introduction.pdf) (högerklicka och spara om den inte öppnas).
___________________________________________________________

## 1.2 Dataanalys med ROOT
Vi importerar C++-biblioteket [ROOT](https://root.cern.ch) som innehåller verktyg för att analysera och visualisera data.

In [2]:
import ROOT

Welcome to JupyROOT 6.14/00


### Vår första loop över kollisioner
Vi hämtar öppet tillgänglig mätdata (sk "open data") från ATLAS-experimentet från internet och laddar in det i ett [ROOT-träd](https://root.cern.ch/doc/master/classTTree.html). Vi börjar med simulerad data, där en $Z$-boson skapats och sönderfallit till ett elektron-positron-par. 

In [3]:
# Open a file with simulated data, containing events with Z bosons decaying to electron-positron pairs (hence the "Zee" in the file name)
f = ROOT.TFile.Open("http://opendata.atlas.cern/release/samples/MC/mc_147770.Zee.root")
tree = f.Get("mini") # get the tree (called "mini") from the file. Every "entry" in a tree represents an event.

#### Trädets innehåll (vad för typ av data finns?)
Innan vi kör vår loop över kollisionerna tittar vi på vilka variabler som finns tillgängliga för varje händelse. Nedan ser ni trädets olika variabler, lagrade som sk grenar (branches). För varje gren listas datatyp och en kort beskrivning. En variabel hämtas med `tree.<branch_name>`. I denna labb ska ni titta på elektroner och/eller myoner, som är leptoner, så variablerna med namn `lep_*` kommer att vara speciellt intressanta.

 <font size="3"><b>branch name</b></font> | <font size="3"><b>type</b></font> | <font size="3"><b>description</b></font> |
 ------------- | :-------------: | :-----: |
 <b id="runNumber">runNumber</b>               | int           | runNumber |
 <b id="eventNumber">eventNumber</b>           | int           | eventNumber |
 <b id="channelNumber">channelNumber</b>       | int           | channelNumber ||
 <b id="mcWeight">mcWeight</b>                 | float         | weight of an MC event |
 <b id="pvxp_n">pvxp_n</b>                     | int           | number of primary vertices |
 <b id="scaleFactor">scaleFactor</b>           | float         | overall scale factor for the preselected event |
 <b id="trigE">trigE</b>                       | bool          | boolean whether a standard trigger has fired in the egamma stream |
 <b id="trigM">trigM</b>                       | bool          | boolean whether a standard trigger has fired in the muon stream |
 <b id="passGRL">passGRL</b>                   | bool          | signifies whether event passes the GRL may be put in isGoodEvent |
 <b id="lep_n">lep_n</b>                       | int           | number of preselected leptons |
 <b id="lep_truthMatched">lep_truthMatched</b> | vector<bool>  | boolean indicating whether the lepton is matched to a truth lepton |
 <b id="lep_trigMatched">lep_trigMatched</b>   | vector<bool>  | boolean signifying whether the lepton is the one triggering the event |
 <b id="lep_pt">lep_pt</b>                    | vector<float> | transverse momentum of the lepton |
 <b id="lep_eta">lep_eta</b>                  | vector<float> | pseudo-rapidity of the lepton |
 <b id="lep_phi">lep_phi</b>                  | vector<float> | azimuthal angle of the lepton |
 <b id="lep_E">lep_E</b>                      | vector<float> | energy of the lepton |
 <b id="lep_z0">lep_z0</b>                    | vector<float> | z-coordinate of the track associated to the lepton wrt. the primary vertex |
 <b id="lep_charge">lep_charge</b>            | vector<float> | charge of the lepton |
 <b id="lep_isTight">lep_isTight</b>          | vector<bool>  | boolean indicating whether the lepton is of tight quality |
 <b id="lep_flag">lep_flag</b>                | vector<int>   | bitmask implementing object cuts of the top group |
 <b id="lep_type">lep_type</b>                | vector<int>   | number signifying the lepton type (e, mu, tau) of the lepton |
 <b id="lep_ptcone30">lep_ptcone30</b>        | vector<float> | ptcone30 isolation for the lepton |
 <b id="lep_etcone20">lep_etcone20</b>        | vector<float> | etcone20 isolation for the lepton |
 <b id="lep_trackd0pvunbiased">lep_trackd0pvunbiased</b> | vector<float>  | d0 of the track associated to the lepton at the point of closest approach (p.o.a.) |
 <b id="lep_tracksigd0pvunbiased">lep_tracksigd0pvunbiased</b> | vector<float>  | d0 signifcance of the track associated to the lepton at the p.o.a. |
 <b id="met_et">met_et                        | float         | Transverse energy of the missing momentum vector |
 <b id="met_phi">met_phi                      | float         | Azimuthal angle of the missing momentum vector |
 <b id="jet_n">jet_n                          | int           | number of selected jets |
 <b id="jet_pt">jet_pt                        | vector<float> | transverse momentum of the jet |
 <b id="jet_eta">jet_eta                      | vector<float> | pseudorapidity of the jet |
 <b id="jet_phi">jet_phi                      | vector<float> | azimuthal angle of the jet |
 <b id="jet_E">jet_E                          | vector<float> | energy of the jet |
 <b id="jet_m">jet_m                          | vector<float> | invariant mass of the jet |
 <b id="jet_jvf">jet_jvf                      | vector<float> | JetVertexFraction of the jet |
 <b id="jet_flag">jet_flag                    | vector<int>   | bitmask implementing object cuts of the top group |
 <b id="jet_trueflav">jet_trueflav            | vector<int>   | true flavor of the jet |
 <b id="jet_truthMatched">jet_truthMatched    | vector<int>   | information whether the jet matches a jet on truth level |
 <b id="jet_SV0">jet_SV0                      | vector<float> | SV0 weight of the jet |
 <b id="jet_MV1">jet_MV1                      | vector<float> | MV1 weight of the jet |
 <b id="scaleFactor_BTAG">scaleFactor_BTAG    | float         | scalefactor for btagging |
 <b id="scaleFactor_ELE">scaleFactor_ELE      | float         | scalefactor for electron efficiency |
 <b id="scaleFactor_JVFSF">scaleFactor_JVFSF  | float         | scalefactor for jet vertex fraction |
 <b id="scaleFactor_MUON">scaleFactor_MUON    | float         | scalefactor for muon efficiency |
 <b id="scaleFactor_PILEUP">scaleFactor_PILEUP   | float      | scalefactor for pileup reweighting |
 <b id="scaleFactor_TRIGGER">scaleFactor_TRIGGER | float      | scalefactor for trigger |
 <b id="scaleFactor_ZVERTEX">scaleFactor_ZVERTEX | float      | scalefactor for z-vertex reweighting |

#### Iterera över de tre första kollisionerna (en liten "event loop")


In [12]:
for ievt in range(3): # let's look closer at the first three events in the tree, indexed 0, 1 and 2
    tree.GetEntry(ievt) # load event with index ievt
    print("Event {}: Number of leptons = {}".format(ievt, tree.lep_n))   #"lep_n" is the number of leptons, by leptons we here mean electrons and muons
    for ilep in range(tree.lep_n): # loop over the leptons in the event
        print("         Lepton {} is of type {} and has a transverse momentum of {:.1f} GeV".format(
            ilep+1, 
            tree.lep_type[ilep], 
            tree.lep_pt[ilep]*1e-3)
        )
print("\nType==11: electron/positron, type==13: muon/anti-muon")

Event 0: Number of leptons = 1
         Lepton 1 is of type 11 and has a transverse momentum of 29.8 GeV
Event 1: Number of leptons = 2
         Lepton 1 is of type 11 and has a transverse momentum of 93.1 GeV
         Lepton 2 is of type 11 and has a transverse momentum of 20.1 GeV
Event 2: Number of leptons = 2
         Lepton 1 is of type 11 and has a transverse momentum of 43.2 GeV
         Lepton 2 is of type 11 and has a transverse momentum of 37.3 GeV

Type==11: electron/positron, type==13: muon/anti-muon


**Tips:** Ett annat sätt att loopa över händelser i ett träd är att skriva `for evt in tree: evt.branch...`. Nedan används detta sätt företrädesvis.

#### Nu testar vi att göra en graf för distributionen av den transversella rörelsemängden för elektroner i de första 1000 händelserna.

In [19]:
# create a histogram with 15 bins, ranging from 15 to 90 GeV
h_pt = ROOT.TH1F("h_pt", "Electron pT; Electron transverse momentum [GeV]; Number of electrons", 15, 15., 90.) 
nevents = 1000
ievt = 0
for evt in tree: # loop over the events
    ievt += 1
    if ievt >= nevents: break # [SWE] avsluta loop efter 1000 händelser
                              # [ENG] terminate the loop after 1000 events
        
    for ilep in range(evt.lep_n):  # [SWE] iterera över leptoner (med leptoner avses elektroner och myoner)
                                   # [ENG] loop over leptons (lepton means electron or muon)
        if evt.lep_type[ilep] == 11: # [SWE] vi vill bara rita elektroner (typ=11)
                                     # [ENG] consider only electrons (type 11)
            h_pt.Fill(evt.lep_pt[ilep]*1e-3) # [SWE] fyll histogram (i filen lagras rörelsemängd i MeV)
                                             # [ENG] fill histogram (file stores momentum in MeV)

Warning in <TDavixFile::Append>: Replacing existing TH1: h_pt (Potential memory leak).


In [15]:
# "magic" command needed for interactive visualisation
%jsroot on 

In [20]:
# open a canvas to draw the histogram on
canvas = ROOT.TCanvas("Canvas", "Title", 800, 600)
# draw the histogram
h_pt.Draw("e1") # draw option "e1" will draw error bars
# open the canvas
canvas.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: Canvas


<b> OBS! </b> Den uppritade figuren är interaktiv: testa att klicka runt (använd högerklick) och scrolla. Ni kan ändra stil och färg på histogrammet, titel på axlarna etc. Figuren kan sparas i png-format genom att högerklicka mitt på canvasen, eller i vektorbaserat PDF-format via kodraden `canvas.SaveAs("figure.pdf")`
___________________________________________________

#### I nästa notebook ska vi lära oss lite mer om hur vi använder ROOT, specifikt om hur man kan anpassa en funktion till uppmätta distributioner för att bestämma parametrar i sin modell.
________________________________________________________